Credit: [DAViD](https://github.com/microsoft/DAViD)

In [ ]:
#@title Install
%cd /content/
!git clone https://github.com/NeuralFalconYT/Remove-Background.git
%cd ./Remove-Background
!apt install aria2 -y
# !aria2c -x 16 -s 16 "https://facesyntheticspubwedata.z6.web.core.windows.net/iccv-2025/models/foreground-segmentation-model-vitb16_384.onnx" #base model
!aria2c -x 16 -s 16 "https://facesyntheticspubwedata.z6.web.core.windows.net/iccv-2025/models/foreground-segmentation-model-vitl16_384.onnx" #large model

!pip install onnxruntime-gpu
# !pip install onnxruntime-gpu==1.22.0
!pip install numpy==2.2.6
!pip install onnx==1.18.0
!pip install opencv-python==4.12.0.88
!pip install gradio>=5.38.2
!pip install gradio_imageslider==0.0.20
from IPython.display import clear_output
clear_output()
import time
time.sleep(5)
import os
os.kill(os.getpid(), 9)


Don't panic if you see this image.png red error cell, that just means Colab automatically restarted the kernel. Now, simply run the next cell.

In [ ]:
%cd /content/Remove-Background
!python app.py